# Задание 1. Шифр Хилла.

### Установка необходимых библиотек:

In [1]:
!pip install numpy sympy


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
from sympy import Matrix
from random import randint

In [3]:
local_alphabet = ['б', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у',
 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', ' ']
n = len(local_alphabet)
print(n)

31


### Ценное сообщение из 12 символов:

In [4]:
message = "я люблю хлеб"

### 3 матрицы-ключа:

In [5]:
key_matrix2 = np.array([[38, 21], [29, 16]])
key_matrix3 = np.array([[17, -39, -9], [-27, -90, 20], [-94, -9, -18]])
key_matrix4 = np.array([[-30, -45, -15, 76], [67, 30, 90, 73], [36, -66, 20, -5], [-97, 26, -20, -69]])

### Убедимся, что определители матриц не имеют общих делителей с n:

In [6]:
det_matrix2 = round(np.linalg.det(key_matrix2))
det_matrix3 = round(np.linalg.det(key_matrix3))
det_matrix4 = round(np.linalg.det(key_matrix4))

print(f"Определители для матриц: 2x2: {det_matrix2}, 3x3: {det_matrix3}, 4x4: {det_matrix4}")
print(f"Условие выполнено: {all(np.gcd(det_matrix_i, n) == 1 for det_matrix_i in (det_matrix2, det_matrix3, det_matrix4))}")

Определители для матриц: 2x2: -1, 3x3: 196827, 4x4: 59833990
Условие выполнено: True


### Зашифруем сообщение с помощью каждого из ключей:

In [7]:
def encrypt(key_matrix, msg):
        message_indices = [local_alphabet.index(char) for char in msg]
        key_size = key_matrix.shape[0]
        
        while len(message_indices) % key_size != 0:
            message_indices.append(local_alphabet.index(" "))

        encrypted_indices = []
        for i in range(0, len(message_indices), key_size):
            block = np.array(message_indices[i : i + key_size])
            encrypted_block = np.dot(key_matrix, block)
            for j in range(len(encrypted_block)):
                if encrypted_block[j] != 0:
                    encrypted_block[j] = encrypted_block[j] % len(local_alphabet)
            encrypted_indices.extend(encrypted_block)
            
        return "".join([local_alphabet[i] for i in encrypted_indices])


enc2, enc3, enc4 = (encrypt(key_matrix_i, message) for key_matrix_i in (key_matrix2, key_matrix3, key_matrix4))
print(f"Сообщения:\n1. {enc2}\n2. {enc3}\n3. {enc4}")

Сообщения:
1. эхбэёццчопрэ
2. уржщпэёънпнж
3. чъёляцешффиы


### Теперь расшифруем сообщения: 

In [8]:
def decrypt(key_matrix, enc):
    dim = key_matrix.shape[0]
    p = []
    dec = ''
    k_inv = np.array(Matrix(key_matrix).inv_mod(n))
    for i in range(len(enc)):
        if i % dim == 0 and i != 0:
            c = k_inv.dot(p) % n
            for j in c:
                dec += local_alphabet[j]
            p = []
        p.append(local_alphabet.index(enc[i]))
    c = k_inv.dot(p) % n
    for j in c:
        dec += local_alphabet[j]
        
    return dec

dec2, dec3, dec4 = (decrypt(key_matrix_i, enc_i) for key_matrix_i, enc_i in zip((key_matrix2, key_matrix3, key_matrix4), (enc2, enc3, enc4)))
print(f"Сообщения:\n1. {dec2}\n2. {dec3}\n3. {dec4}")

Сообщения:
1. я люблю хлеб
2. я люблю хлеб
3. я люблю хлеб


### Сымитируем вредоносное вмешательство в зашифрованные сообщения:

In [9]:
def hack(enc):
    indexes = []
    chars = []
    listed_mes = list(enc)

    while len(indexes) != 3:
        index = np.random.randint(0, len(enc))
        char = local_alphabet[np.random.randint(0, n)]
        if (index not in indexes) and (char != enc[index]):
            indexes.append(index)
            chars.append(char)

    for i in range(0, 3):
        listed_mes[indexes[i]] = chars[i]

    return "".join(listed_mes)


hacked_enc2, hacked_enc3, hacked_enc4 = (hack(enc_i) for enc_i in (enc2, enc3, enc4))
print(f"Сообщения:\n1. {hacked_enc2}\n2. {hacked_enc3}\n3. {hacked_enc4}")

Сообщения:
1. эхбюёпцчопйэ
2. сржщпьёънжнж
3. чъёляци ф иы


### Расшифруем получившиеся сообщения: 

In [10]:
hacked_dec2, hacked_dec3, hacked_dec4 = (decrypt(key_matrix_i, enc_i) for key_matrix_i, enc_i in zip((key_matrix2, key_matrix3, key_matrix4), (hacked_enc2, hacked_enc3, hacked_enc4)))

print(f"Сообщения:\n1. {hacked_dec2}\n2. {hacked_dec3}\n3. {hacked_dec4}")

Сообщения:
1. я  чкэю хлчр
2. пуыетфю х чм
3. я люймщёрыот


# Задание 2. Взлом шифра Хилла.

### Ключ размера 2x2:

In [11]:
matrix_key = np.array([[1, 3], [3, 1]])

### 2 сообщения из 12 символов:

In [12]:
message1 = "я люблю хлеб"
message2 = "я люблю борщ"

### Шифруем 2 сообщения: 

In [13]:
enc1, enc2 = (encrypt(matrix_key, message_i) for message_i in (message1, message2))
print(f"Сообщения:\n1. {enc1}\n2. {enc2}")

Сообщения:
1. ьъбъэлычсжеи
2. ьъбъэлычзочё


### Забыли одно исходное сообщение:

In [14]:
message1 = None

### Восстанавливаем ключ по исходному и шифрованному сообщению:

In [15]:
p, c = [], []
P, C = [], []
for i in range(4):
    p.append(local_alphabet.index(message2[i]))
    c.append(local_alphabet.index(enc2[i]))
    if len(p) == 2:
        P.append(p)
        C.append(c)
        p, c = [], []
P = np.array(P)
C = np.array(C)
matrix_key = np.array((Matrix(P).inv_mod(n)  * Matrix(C)) % n, dtype=int)

### Расшифровываем забытое сообщение:

In [16]:
decrypt(matrix_key, enc1)

'я люблю хлеб'

# Задание 3. Код Хэмминга

### Русский алфавит:


In [17]:
russian_alphabet = ['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н','о', 'п', 'р', 'с', 'т', 'у', 'ф','х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э','ю','я']
n = len(russian_alphabet)
n

32

### Присвоим каждой букве пятибитовый двоичный номер:

In [18]:
def convert_to_five_bit_number(number):
    bin_number = ""
    while number > 0:
        bin_number += str(number % 2)
        number //= 2
    return "0" * (5 - len(bin_number)) + bin_number[::-1]


russian_bin_alphabet = {russian_alphabet[i]: convert_to_five_bit_number(i) for i in range(n)}
russian_bin_alphabet

{'а': '00000',
 'б': '00001',
 'в': '00010',
 'г': '00011',
 'д': '00100',
 'е': '00101',
 'ж': '00110',
 'з': '00111',
 'и': '01000',
 'й': '01001',
 'к': '01010',
 'л': '01011',
 'м': '01100',
 'н': '01101',
 'о': '01110',
 'п': '01111',
 'р': '10000',
 'с': '10001',
 'т': '10010',
 'у': '10011',
 'ф': '10100',
 'х': '10101',
 'ц': '10110',
 'ч': '10111',
 'ш': '11000',
 'щ': '11001',
 'ъ': '11010',
 'ы': '11011',
 'ь': '11100',
 'э': '11101',
 'ю': '11110',
 'я': '11111'}

### Интересное слово из 4 букв:

In [19]:
word = "шифр"

### Закодируем двоичным кодом:

In [20]:
encrypted_bin_word = "".join([russian_bin_alphabet[letter] for letter in list(word)])
encrypted_bin_word

'11000010001010010000'

### Составим матрицы G и H:

In [21]:
G = np.array([[1, 1, 0, 1], [1, 0, 1, 1], [1, 0, 0, 0], [0, 1, 1, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
H = np.array([[1, 0, 1, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, 1]])

* Матрица G - это матрица-генератор кодов, которая преобразует информационное сообщение, представленное вектором размерности (4,1), в кодовое слово размерности (7,1). При этом она добавляет три дополнительных бита четности. Поэтому размер матрицы G составляет (4,7).
* Матрица H используется для проверки целостности закодированного сообщения. Она анализирует кодовое слово размерности (7,1) и возвращает вектор размерности (3,1), который указывает на наличие ошибок. Если кодовое слово не повреждено, результат будет нуль-вектором; если присутствует ошибка, матрица H укажет на позицию испорченного бита. Соответственно, размер матрицы H составляет (3,7).
* Можно сделать вывод, что ker H = Im G

### Закодируем слово с помощью матрицы G:

In [22]:
encoded_word = ''
for i in range(0, len(encrypted_bin_word), 4):
    p = np.array([int(c) for c in encrypted_bin_word[i : i + 4]])
    x = G.dot(p) % 2
    encoded_word += ''.join([str(i) for i in x])

encoded_word

'01111000101010010101000110010000000'

### Сымитируем вредоносное вмешательство:

In [23]:
broken_words = []
broken1 = list(encoded_word)
for _ in range(1):
    i = randint(0, len(broken1) - 1)
    broken1[i] = '1' if broken1[i] == '0' else '0'
broken1 = ''.join(broken1)
broken_words.append(broken1)

broken2 = list(encoded_word)
for _ in range(2):
    i = randint(0, len(broken2) - 1)
    broken2[i] = '1' if broken2[i] == '0' else '0'
broken2 = ''.join(broken2)
broken_words.append(broken2)

broken3 = list(encoded_word)
for _ in range(3):
    i = randint(0, len(broken3) - 1)
    broken3[i] = '1' if broken3[i] == '0' else '0'
broken3 = ''.join(broken3)
broken_words.append(broken3)

broken4 = list(encoded_word)
for _ in range(4):
    i = randint(0, len(broken4) - 1)
    broken4[i] = '1' if broken4[i] == '0' else '0'
broken4 = ''.join(broken4)
broken_words.append(broken4)

### Находим и исправляем ошибки с помощью матрицы H и декодируем сообщение:

In [24]:
R = np.array([[0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]])
word_bins = []
for broken in broken_words:
    word_bin = ''
    for i in range(0, len(broken), 7):
        r = np.array([int(c) for c in broken[i : i + 7]])
        z = H.dot(r) % 2
        print(z)
        if sum(z) != 0:
            idx = int(''.join([str(n) for n in z[::-1]]), 2) - 1
            r[idx] = 1 if r[idx] == 0 else 0
        p = R.dot(r) % 2
        word_bin += ''.join([str(i) for i in p])
    print('---')
    word_bins.append(word_bin)

[0 1 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
---
[0 0 0]
[1 1 1]
[0 0 0]
[0 0 0]
[0 0 0]
---
[0 0 0]
[1 1 1]
[1 1 1]
[1 0 0]
[0 0 0]
---
[1 0 1]
[0 0 0]
[1 1 0]
[0 0 1]
[0 0 0]
---


### Переводим полученный результат в слово из 4-х букв

In [25]:
for word_bin in word_bins:
    result = ''
    for i in range(0, len(word_bin), 5):
        result += russian_alphabet[list(russian_bin_alphabet.values()).index(word_bin[i : i + 5])]
    print(result)

шифр
щмфр
шифр
рифр


#### Вывод: даже не зная ключа, но имея оригинал и зашифрованное сообщение, можно установить закономерности шифра, определить матрицу-ключ и взломать шифр, что позволит в дальнейшем расшифровывать любые зашифрованные сообщения.